In [1]:
from google.colab import drive
drive.mount('/tfcontent')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /tfcontent


In [2]:
import pandas as pd 
import numpy as np
data=pd.read_csv(r'/tfcontent/My Drive/yelp.csv')
data.head()

,stars,text
0,5,My wife took me here on my birthday for breakf...
1,5,I have no idea why some people give bad review...
2,4,love the gyro plate. Rice is so good and I als...
3,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!..."
4,5,General Manager Scott Petello is a good egg!!!...


In [3]:
x=data['text']
y=data['stars']
print(x.shape)
print(y.shape)

(10000,)
(10000,)


In [4]:
data['Label']=['1' if star > 3 else '0' for star in data['stars']];
data.head()

,stars,text,Label
0,5,My wife took me here on my birthday for breakf...,1
1,5,I have no idea why some people give bad review...,1
2,4,love the gyro plate. Rice is so good and I als...,1
3,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",1
4,5,General Manager Scott Petello is a good egg!!!...,1


In [6]:
y=data['Label']
y.head()

0    1
1    1
2    1
3    1
4    1
Name: Label, dtype: object

In [9]:
import nltk
nltk.download('stopwords')
# import string # importing string bcz was getting string error while joining sentences
stop = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
def clean_document(doco):
    punctuation = string.punctuation
    punc_replace = ''.join([' ' for s in punctuation])
    doco_link_clean = re.sub(r'http\S+', '', doco)
    doco_clean_and = re.sub(r'&\S+', '', doco_link_clean)
    doco_clean_at = re.sub(r'@\S+', '', doco_clean_and)
    doco_clean = doco_clean_at.replace('-', ' ')
    doco_alphas = re.sub(r'\W +', ' ', doco_clean)
    trans_table = str.maketrans(punctuation, punc_replace)
    doco_clean = ' '.join([word.translate(trans_table) for word in doco_alphas.split(' ')])
    doco_clean = doco_clean.split(' ')
    p = re.compile(r'\s*\b(?=[a-z\d]*([a-z\d])\1{3}|\d+\b)[a-z\d]+', re.IGNORECASE)
    doco_clean = ([p.sub("", x).strip() for x in doco_clean])
    doco_clean = [word.lower() for word in doco_clean if len(word) > 2]
    doco_clean = ([i for i in doco_clean if i not in stop])
#     doco_clean = [spell(word) for word in doco_clean]
#     p = re.compile(r'\s*\b(?=[a-z\d]*([a-z\d])\1{3}|\d+\b)[a-z\d]+', re.IGNORECASE)
    doco_clean = ([p.sub("", x).strip() for x in doco_clean])
#     doco_clean = ([spell(k) for k in doco_clean])
    return doco_clean

In [0]:
import re # bcz was getting NameError: global name 're' is not defined
import string # importing string bcz was getting string error while joining sentences

In [0]:
cleaned_reviews = [clean_document(doc) for doc in data['text']];
sentences = [' '.join(r) for r in cleaned_reviews ] #joining

In [13]:
print(x[12],'\n')

print(sentences[12])

They have a limited time thing going on right now with BBQ chicken pizza (not sure how long it's going to last) but let me just say it was amazing.  Probably THE best BBQ Chicken pizza I have ever had.  I have tried other things too, like the tomato basil soup, and many of their sandwiches ... very good, very fresh - every time.  

The 5 stars is for the pizza, but if I were to rate Jason's Deli over all they would get about a 4. 

limited time thing going right bbq chicken pizza sure long going last let say amazing probably best bbq chicken pizza ever tried things like tomato basil soup many sandwiches good fresh every time stars pizza rate jason deli would get


In [15]:
data['cleaned_text'] = sentences
data.head(5)


,stars,text,Label,cleaned_text
0,5,My wife took me here on my birthday for breakf...,1,wife took birthday breakfast excellent weather...
1,5,I have no idea why some people give bad review...,1,idea people give bad reviews place goes show p...
2,4,love the gyro plate. Rice is so good and I als...,1,love gyro plate rice good also dig candy selec...
3,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",1,rosie dakota love chaparral dog park convenien...
4,5,General Manager Scott Petello is a good egg!!!...,1,general manager scott petello good egg detail ...


In [0]:
#Bag of words
from sklearn.feature_extraction.text import CountVectorizer#CountVectorizer is used to create bag of words which will help machine learn whether a certan word is good or bad
bow_transformer = CountVectorizer(analyzer=clean_document).fit(x)

In [0]:
X = bow_transformer.transform(x) # if we don't use transform function that we get ValueError: could not convert string to float:

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [0]:
#Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier
model= GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
model.fit(X_train, y_train)
predicted= model.predict(X_test)

In [20]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y_test, predicted))
print('\n')
print(classification_report(y_test, predicted))
model.score(X_train,y_train)

[[ 483  453]
 [ 202 1862]]


             precision    recall  f1-score   support

          0       0.71      0.52      0.60       936
          1       0.80      0.90      0.85      2064

avg / total       0.77      0.78      0.77      3000



0.8314285714285714

In [24]:
print(model.predict(bow_transformer.transform(['love the gyro plate rice is so good also dig their candy selection '])))

['1']


In [27]:
print(model.predict(bow_transformer.transform(['bad food, not upto the mark'])))

['0']


In [28]:
#KNN Classifier
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [29]:
predsknn=neigh.predict(X_test)

from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y_test, predsknn))
print('\n')
print(classification_report(y_test, predsknn))
neigh.score(X_train,y_train)

[[ 545  391]
 [ 152 1912]]


             precision    recall  f1-score   support

          0       0.78      0.58      0.67       936
          1       0.83      0.93      0.88      2064

avg / total       0.82      0.82      0.81      3000



0.8184285714285714

In [31]:
print(neigh.predict(bow_transformer.transform(['Very healthy and delicious food, love the beer too, Apple Cider was my favourite :)'])))

['1']


In [40]:
print(neigh.predict(bow_transformer.transform(['not happy with anything'])))

['0']


In [0]:
from sklearn.linear_model import LogisticRegression
#create a model
classifier=(LogisticRegression())
#fitting training data to  the model
classifier.fit(X_train,y_train)
Y_pred=classifier.predict(X_test)
# print(list(zip(y_test,Y_pred)))

In [70]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

cfm=confusion_matrix(y_test,Y_pred)
print(cfm)

print("Classification report:")

print(classification_report(y_test,Y_pred))

acc=accuracy_score(y_test,Y_pred)
print("Accuracy of the model:",acc)

[[ 628  308]
 [ 256 1808]]
Classification report:
             precision    recall  f1-score   support

          0       0.71      0.67      0.69       936
          1       0.85      0.88      0.87      2064

avg / total       0.81      0.81      0.81      3000

Accuracy of the model: 0.812


In [73]:
print(classifier.predict(bow_transformer.transform(['Glad I came, food was good loved the fact that it is a pet friendly place also the beer was amazing'])))

['1']


In [74]:
print(classifier.predict(bow_transformer.transform(['ambiance was bad, disliked everything']))) 

['0']
